In [1]:
from pydantic import BaseModel, Field

from qdrant_client import QdrantClient
from qdrant_client.models import Prefetch, Filter, FieldCondition, MatchAny, FusionQuery, Document


from langsmith import traceable, get_current_run_tree

from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode
from langgraph.types import Send, Command

from langchain_core.messages import AIMessage, ToolMessage, convert_to_openai_messages

from jinja2 import Template
from typing import Literal, Dict, Any, Annotated, List, Optional, Sequence
from IPython.display import Image, display
from operator import add
from openai import OpenAI

import openai

import random
import ast
import inspect
import instructor
import json
import numpy as np
import psycopg2
from psycopg2.extras import RealDictCursor, execute_batch

from utils.utils import get_tool_descriptions, format_ai_message
from utils.tools import get_shopping_cart, add_to_shopping_cart, remove_from_cart, get_formatted_item_context, get_formatted_reviews_context

from langgraph.checkpoint.postgres import PostgresSaver


/Users/tanyakanodia/learning-ai/00-ai-engineering-bootcamp-cohort-2/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Fictional warehouses

In [2]:
warehouses = [
    {
        "warehouse_id": "DE-BER-01",
        "warehouse_location": "Berlin, Germany",
        "warehouse_name": "Berlin Distribution Center"
    },
    {
        "warehouse_id": "DE-MUN-01",
        "warehouse_location": "Munich, Germany",
        "warehouse_name": "Munich Logistics Hub"
    },
    {
        "warehouse_id": "DE-HAM-01",
        "warehouse_location": "Hamburg, Germany",
        "warehouse_name": "Hamburg North Warehouse"
    },
    {
        "warehouse_id": "FR-PAR-01",
        "warehouse_location": "Paris, France",
        "warehouse_name": "Paris Central Depot"
    },
    {
        "warehouse_id": "FR-LYO-01",
        "warehouse_location": "Lyon, France",
        "warehouse_name": "Lyon Regional Warehouse"
    },
    {
        "warehouse_id": "FR-MAR-01",
        "warehouse_location": "Marseille, France",
        "warehouse_name": "Marseille Mediterranean Hub"
    }
]

### Simulate Stock Availability for each of the warehouse
##### Retrieve all item IDs from Amazon Items Qdrant Collection

In [3]:
qdrant_client = QdrantClient(url="http://localhost:6333")
dummy_vector = np.zeros(1536).tolist()

In [4]:
payload = qdrant_client.query_points(
    collection_name="Amazon-items-collection-hybrid-search",
    query=dummy_vector,
    using="text-embedding-3-small",
    limit=1000,
    with_payload=["parent_asin"],
    with_vectors=False
)

In [9]:
payload.points




[ScoredPoint(id=347, version=2, score=0.0, payload={'parent_asin': 'B09V7LNPC3'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=430, version=2, score=0.0, payload={'parent_asin': 'B0C7CPL179'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=361, version=2, score=0.0, payload={'parent_asin': 'B09LH466KZ'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=278, version=2, score=0.0, payload={'parent_asin': 'B0B2P18Z31'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=450, version=2, score=0.0, payload={'parent_asin': 'B09V5D3B17'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=461, version=2, score=0.0, payload={'parent_asin': 'B0B17SD56J'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=493, version=2, score=0.0, payload={'parent_asin': 'B0BSRGYNWV'}, vector=None, shard_key=None, order_value=None),
 ScoredPoint(id=79, version=2, score=0.0, payload={'parent_asin': 'B09GNF94GQ'}, vector=No

In [8]:
parent_asin_list = [item.payload["parent_asin"] for item in payload.points]
parent_asin_list

['B09V7LNPC3',
 'B0C7CPL179',
 'B09LH466KZ',
 'B0B2P18Z31',
 'B09V5D3B17',
 'B0B17SD56J',
 'B0BSRGYNWV',
 'B09GNF94GQ',
 'B0BJJYFRLF',
 'B0BJZV7QVF',
 'B09W5RJ1LY',
 'B0BR33XH8D',
 'B0B8TH21GK',
 'B0B4915S57',
 'B09SHPX1FQ',
 'B0BTD88265',
 'B0BP68QCZ2',
 'B0C7GS19F2',
 'B09R1TMXR3',
 'B0BBG6P5SM',
 'B0BF18F6R7',
 'B0C6K1GQCF',
 'B09V18BFYL',
 'B0BYSY8NXV',
 'B0C1NDPD5L',
 'B0BQYN54PZ',
 'B09SD9NF6N',
 'B0BZB3K3M5',
 'B0C48WYVBQ',
 'B0BN8Q1Z9S',
 'B0B2V9P3K4',
 'B07MZK432D',
 'B0CDWZPRF2',
 'B09TKVV8MJ',
 'B0BT344Q4K',
 'B09WR4KR4X',
 'B0BTPG4XHH',
 'B0BX85LFW8',
 'B0BSF9NZ3V',
 'B0BVZ47N6N',
 'B0BWDRP69M',
 'B09QGNB537',
 'B09V9P7Z1T',
 'B0BCCCNHD8',
 'B0BMG766L1',
 'B0C3XS6N81',
 'B09FFFSH25',
 'B0BBPTY7Y7',
 'B09SHNKDBC',
 'B0BHDX2GDZ',
 'B09QLKN43H',
 'B0C5WYSL3B',
 'B0B8DJYF59',
 'B09XX27FH2',
 'B0BPNNV5NW',
 'B09R8YGSLN',
 'B0B9JF64DM',
 'B0BL2CZSHT',
 'B09ZV2XJ65',
 'B0BDQSK1JG',
 'B09GNFDH9M',
 'B09QJRYJW4',
 'B0B61Z8LKM',
 'B0BR8Q2PXN',
 'B09PQ97H3G',
 'B0B5W5NKKN',
 'B0B14HTZ

In [7]:
len(parent_asin_list)

500

### Populate availability database for all items in qdrant db - this is a synthetic quantities population / generation for our usecase

In [10]:
def generate_inventory_data(warehouses, product_ids, availability_rate=0.75):
    
    inventory_records = []
    
    for warehouse in warehouses:
        for product_id in product_ids:
            # 75% chance the product is available in this warehouse
            if random.random() < availability_rate:
                total_quantity = random.randint(0, 100)
                
                # Only add to inventory if quantity > 0
                if total_quantity > 0:
                    inventory_records.append({
                        "warehouse_id": warehouse["warehouse_id"],
                        "warehouse_location": warehouse["warehouse_location"],
                        "warehouse_name": warehouse["warehouse_name"],
                        "product_id": product_id,
                        "total_quantity": total_quantity,
                        "reserved_quantity": 0  # Starting with no reservations
                    })
    
    return inventory_records

In [11]:
inventory_data = generate_inventory_data(warehouses, parent_asin_list, availability_rate=0.75)
inventory_data

[{'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B09V7LNPC3',
  'total_quantity': 95,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0C7CPL179',
  'total_quantity': 85,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0B2P18Z31',
  'total_quantity': 62,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B09V5D3B17',
  'total_quantity': 92,
  'reserved_quantity': 0},
 {'warehouse_id': 'DE-BER-01',
  'warehouse_location': 'Berlin, Germany',
  'warehouse_name': 'Berlin Distribution Center',
  'product_id': 'B0B17SD56J',
  'total_quantity': 59,
  

In [12]:
len(inventory_data)

2247

In [13]:
3000*0.75

2250.0

### Write synthetic data to Postgres

In [14]:
def insert_inventory_to_db(inventory_records):
   
    try:
        # Connect to the database
        conn = psycopg2.connect(
            host="localhost",
            port=5433,
            database="langgraph_db",
            user="langgraph_user",
            password="langgraph_password"
        )
        conn.autocommit = True

        with conn.cursor(cursor_factory=RealDictCursor) as cursor:
        
            # Prepare the INSERT query
            insert_query = """
            INSERT INTO warehouses.inventory 
            (warehouse_id, warehouse_location, warehouse_name, product_id, total_quantity, reserved_quantity)
            VALUES (%(warehouse_id)s, %(warehouse_location)s, %(warehouse_name)s, %(product_id)s, %(total_quantity)s, %(reserved_quantity)s)
            """
            
            # Use execute_batch for better performance with many inserts
            execute_batch(cursor, insert_query, inventory_records, page_size=100)
            
            # Commit the transaction
            conn.commit()
            
            print(f"Successfully inserted {len(inventory_records)} records into warehouses.inventory")
            
            # Close cursor and connection
            cursor.close()
            conn.close()
        
    except psycopg2.Error as e:
        print(f"Database error: {e}")
        if conn:
            conn.rollback()
    except Exception as e:
        print(f"Error: {e}")
    finally:
        if cursor:
            cursor.close()
        if conn:
            conn.close()

In [15]:
insert_inventory_to_db(inventory_data)


Successfully inserted 2247 records into warehouses.inventory
